In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW38OT-MakeConcordance-Zechariah.py
"""
Created on Thu Jul 14 22:43:17 2016

@author: keith
"""

from BibleWordList2 import *

Zechariah01 = [
['Zechariah','Zech','01','001','1',"In the eighth month, in the second year of Darius, came the word of the LORD unto Zechariah, the son of Berechiah, the son of Iddo the prophet, saying,"],
['Zechariah','Zech','01','002','2',"The LORD hath been sore displeased with your fathers."],
['Zechariah','Zech','01','003','3',"Therefore say thou unto them, Thus saith the LORD of hosts; Turn ye unto me, saith the LORD of hosts, and I will turn unto you, saith the LORD of hosts."],
['Zechariah','Zech','01','004','4',"Be ye not as your fathers, unto whom the former prophets have cried, saying, Thus saith the LORD of hosts; Turn ye now from your evil ways, and \emph{from} your evil doings: but they did not hear, nor hearken unto me, saith the LORD."],
['Zechariah','Zech','01','005','5',"Your fathers, where \emph{are} they? and the prophets, do they live for ever?"],
['Zechariah','Zech','01','006','6',"But my words and my statutes, which I commanded my servants the prophets, did they not take hold of your fathers? and they returned and said, Like as the LORD of hosts thought to do unto us, according to our ways, and according to our doings, so hath he dealt with us."],
['Zechariah','Zech','01','007','7',"Upon the four and twentieth day of the eleventh month, which \emph{is} the month Sebat, in the second year of Darius, came the word of the LORD unto Zechariah, the son of Berechiah, the son of Iddo the prophet, saying,"],
['Zechariah','Zech','01','008','8',"I saw by night, and behold a man riding upon a red horse, and he stood among the myrtle trees that \emph{were} in the bottom; and behind him \emph{were} \emph{there} red horses, speckled, and white."],
['Zechariah','Zech','01','009','9',"Then said I, O my lord, what \emph{are} these? And the angel that talked with me said unto me, I will shew thee what these \emph{be}."],
['Zechariah','Zech','01','010','10',"And the man that stood among the myrtle trees answered and said, These \emph{are} \emph{they} whom the LORD hath sent to walk to and fro through the earth."],
['Zechariah','Zech','01','011','11',"And they answered the angel of the LORD that stood among the myrtle trees, and said, We have walked to and fro through the earth, and, behold, all the earth sitteth still, and is at rest."],
['Zechariah','Zech','01','012','12',"Then the angel of the LORD answered and said, O LORD of hosts, how long wilt thou not have mercy on Jerusalem and on the cities of Judah, against which thou hast had indignation these threescore and ten years?"],
['Zechariah','Zech','01','013','13',"And the LORD answered the angel that talked with me \emph{with} good words \emph{and} comfortable words."],
['Zechariah','Zech','01','014','14',"So the angel that communed with me said unto me, Cry thou, saying, Thus saith the LORD of hosts; I am jealous for Jerusalem and for Zion with a great jealousy."],
['Zechariah','Zech','01','015','15',"And I am very sore displeased with the heathen \emph{that} \emph{are} at ease: for I was but a little displeased, and they helped forward the affliction."],
['Zechariah','Zech','01','016','16',"Therefore thus saith the LORD; I am returned to Jerusalem with mercies: my house shall be built in it, saith the LORD of hosts, and a line shall be stretched forth upon Jerusalem."],
['Zechariah','Zech','01','017','17',"Cry yet, saying, Thus saith the LORD of hosts; My cities through prosperity shall yet be spread abroad; and the LORD shall yet comfort Zion, and shall yet choose Jerusalem."],
['Zechariah','Zech','01','018','18',"Then lifted I up mine eyes, and saw, and behold four horns."],
['Zechariah','Zech','01','019','19',"And I said unto the angel that talked with me, What \emph{be} these? And he answered me, These \emph{are} the horns which have scattered Judah, Israel, and Jerusalem."],
['Zechariah','Zech','01','020','20',"And the LORD shewed me four carpenters."],
['Zechariah','Zech','01','021','21',"Then said I, What come these to do? And he spake, saying, These \emph{are} the horns which have scattered Judah, so that no man did lift up his head: but these are come to fray them, to cast out the horns of the Gentiles, which lifted up \emph{their} horn over the land of Judah to scatter it."]
]

Zechariah02 = [
['Zechariah','Zech','02','001','1',"I lifted up mine eyes again, and looked, and behold a man with a measuring line in his hand."],
['Zechariah','Zech','02','002','2',"Then said I, Whither goest thou? And he said unto me, To measure Jerusalem, to see what \emph{is} the breadth thereof, and what \emph{is} the length thereof."],
['Zechariah','Zech','02','003','3',"And, behold, the angel that talked with me went forth, and another angel went out to meet him,"],
['Zechariah','Zech','02','004','4',"And said unto him, Run, speak to this young man, saying, Jerusalem shall be inhabited \emph{as} towns without walls for the multitude of men and cattle therein:"],
['Zechariah','Zech','02','005','5',"For I, saith the LORD, will be unto her a wall of fire round about, and will be the glory in the midst of her."],
['Zechariah','Zech','02','006','6',"Ho, ho, \emph{come} \emph{forth}, and flee from the land of the north, saith the LORD: for I have spread you abroad as the four winds of the heaven, saith the LORD."],
['Zechariah','Zech','02','007','7',"Deliver thyself, O Zion, that dwellest \emph{with} the daughter of Babylon."],
['Zechariah','Zech','02','008','8',"For thus saith the LORD of hosts; After the glory hath he sent me unto the nations which spoiled you: for he that toucheth you toucheth the apple of his eye."],
['Zechariah','Zech','02','009','9',"For, behold, I will shake mine hand upon them, and they shall be a spoil to their servants: and ye shall know that the LORD of hosts hath sent me."],
['Zechariah','Zech','02','010','10',"Sing and rejoice, O daughter of Zion: for, lo, I come, and I will dwell in the midst of thee, saith the LORD."],
['Zechariah','Zech','02','011','11',"And many nations shall be joined to the LORD in that day, and shall be my people: and I will dwell in the midst of thee, and thou shalt know that the LORD of hosts hath sent me unto thee."],
['Zechariah','Zech','02','012','12',"And the LORD shall inherit Judah his portion in the holy land, and shall choose Jerusalem again."],
['Zechariah','Zech','02','013','13',"Be silent, O all flesh, before the LORD: for he is raised up out of his holy habitation."]
]


Zechariah03 = [
['Zechariah','Zech','03','001','1',"And he shewed me Joshua the high priest standing before the angel of the LORD, and Satan standing at his right hand to resist him."],
['Zechariah','Zech','03','002','2',"And the LORD said unto Satan, The LORD rebuke thee, O Satan; even the LORD that hath chosen Jerusalem rebuke thee: \emph{is} not this a brand plucked out of the fire?"],
['Zechariah','Zech','03','003','3',"Now Joshua was clothed with filthy garments, and stood before the angel."],
['Zechariah','Zech','03','004','4',"And he answered and spake unto those that stood before him, saying, Take away the filthy garments from him. And unto him he said, Behold, I have caused thine iniquity to pass from thee, and I will clothe thee with change of raiment."],
['Zechariah','Zech','03','005','5',"And I said, Let them set a fair mitre upon his head. So they set a fair mitre upon his head, and clothed him with garments. And the angel of the LORD stood by."],
['Zechariah','Zech','03','006','6',"And the angel of the LORD protested unto Joshua, saying,"],
['Zechariah','Zech','03','007','7',"Thus saith the LORD of hosts; If thou wilt walk in my ways, and if thou wilt keep my charge, then thou shalt also judge my house, and shalt also keep my courts, and I will give thee places to walk among these that stand by."],
['Zechariah','Zech','03','008','8',"Hear now, O Joshua the high priest, thou, and thy fellows that sit before thee: for they \emph{are} men wondered at: for, behold, I will bring forth my servant the BRANCH."],
['Zechariah','Zech','03','009','9',"For behold the stone that I have laid before Joshua; upon one stone \emph{shall} \emph{be} seven eyes: behold, I will engrave the graving thereof, saith the LORD of hosts, and I will remove the iniquity of that land in one day."],
['Zechariah','Zech','03','010','10',"In that day, saith the LORD of hosts, shall ye call every man his neighbour under the vine and under the fig tree."]
]

Zechariah04 = [
    ['Zechariah','Zech','04','01','1',"And the angel that talked with me came again, and waked me, as a man that is wakened out of his sleep,"],
    ['Zechariah','Zech','04','02','2',"And said unto me, What seest thou? And I said, I have looked, and behold a candlestick all \emph{of} gold, with a bowl upon the top of it, and his seven lamps thereon, and seven pipes to the seven lamps, which \emph{are} upon the top thereof:"],
    ['Zechariah','Zech','04','03','3',"And two olive trees by it, one upon the right \emph{side} of the bowl, and the other upon the left \emph{side} thereof."],
    ['Zechariah','Zech','04','04','4',"So I answered and spake to the angel that talked with me, saying, What \emph{are} these, my lord?"],
    ['Zechariah','Zech','04','05','5',"Then the angel that talked with me answered and said unto me, Knowest thou not what these be? And I said, No, my lord."],
    ['Zechariah','Zech','04','06','6',"Then he answered and spake unto me, saying, This \emph{is} the word of the LORD unto Zerubbabel, saying, Not by might, nor by power, but by my spirit, saith the LORD of hosts."],
    ['Zechariah','Zech','04','07','7',"Who \emph{art} thou, O great mountain? before Zerubbabel \emph{thou} \emph{shalt} \emph{become} a plain: and he shall bring forth the headstone \emph{thereof} \emph{with} shoutings, \emph{crying}, Grace, grace unto it."],
    ['Zechariah','Zech','04','08','8',"Moreover the word of the LORD came unto me, saying,"],
    ['Zechariah','Zech','04','09','9',"The hands of Zerubbabel have laid the foundation of this house; his hands shall also finish it; and thou shalt know that the LORD of hosts hath sent me unto you."],
    ['Zechariah','Zech','04','10','10',"For who hath despised the day of small things? for they shall rejoice, and shall see the plummet in the hand of Zerubbabel \emph{with} those seven; they \emph{are} the eyes of the LORD, which run to and fro through the whole earth."],
    ['Zechariah','Zech','04','11','11',"Then answered I, and said unto him, What \emph{are} these two olive trees upon the right \emph{side} of the candlestick and upon the left \emph{side} thereof?"],
    ['Zechariah','Zech','04','12','12',"And I answered again, and said unto him, What \emph{be} \emph{these} two olive branches which through the two golden pipes empty the golden \emph{oil} out of themselves?"],
    ['Zechariah','Zech','04','13','13',"And he answered me and said, Knowest thou not what these \emph{be}? And I said, No, my lord."],
    ['Zechariah','Zech','04','14','14',"Then said he, These \emph{are} the two anointed ones, that stand by the Lord of the whole earth."]]

Zechariah05 = [
    ['Zechariah','Zech','05','01','1',"Then I turned, and lifted up mine eyes, and looked, and behold a flying roll."],
    ['Zechariah','Zech','05','02','2',"And he said unto me, What seest thou? And I answered, I see a flying roll; the length thereof \emph{is} twenty cubits, and the breadth thereof ten cubits."],
    ['Zechariah','Zech','05','03','3',"Then said he unto me, This \emph{is} the curse that goeth forth over the face of the whole earth: for every one that stealeth shall be cut off \emph{as} on this side according to it; and every one that sweareth shall be cut off \emph{as} on that side according to it."],
    ['Zechariah','Zech','05','04','4',"I will bring it forth, saith the LORD of hosts, and it shall enter into the house of the thief, and into the house of him that sweareth falsely by my name: and it shall remain in the midst of his house, and shall consume it with the timber thereof and the stones thereof."],
    ['Zechariah','Zech','05','05','5',"Then the angel that talked with me went forth, and said unto me, Lift up now thine eyes, and see what \emph{is} this that goeth forth"],
    ['Zechariah','Zech','05','06','6',"And I said, What \emph{is} it? And he said, This \emph{is} an ephah that goeth forth. He said moreover, This \emph{is} their resemblance through all the earth."],
    ['Zechariah','Zech','05','07','7',"And, behold, there was lifted up a talent of lead: and this \emph{is} a woman that sitteth in the midst of the ephah."],
    ['Zechariah','Zech','05','08','8',"And he said, This \emph{is} wickedness. And he cast it into the midst of the ephah; and he cast the weight of lead upon the mouth thereof."],
    ['Zechariah','Zech','05','09','9',"Then lifted I up mine eyes, and looked, and, behold, there came out two women, and the wind \emph{was} in their wings; for they had wings like the wings of a stork: and they lifted up the ephah between the earth and the heaven."],
    ['Zechariah','Zech','05','10','10',"Then said I to the angel that talked with me, Whither do these bear the ephah?"],
    ['Zechariah','Zech','05','11','11',"And he said unto me, To build it an house in the land of Shinar: and it shall be established, and set there upon her own base."]]

Zechariah06 = [
['Zechariah','Zech','06','001','1',"And I turned, and lifted up mine eyes, and looked, and, behold, there came four chariots out from between two mountains; and the mountains \emph{were} mountains of brass."],
['Zechariah','Zech','06','002','2',"In the first chariot \emph{were} red horses; and in the second chariot black horses;"],
['Zechariah','Zech','06','003','3',"And in the third chariot white horses; and in the fourth chariot grisled and bay horses."],
['Zechariah','Zech','06','004','4',"Then I answered and said unto the angel that talked with me, What \emph{are} these, my lord?"],
['Zechariah','Zech','06','005','5',"And the angel answered and said unto me, These \emph{are} the four spirits of the heavens, which go forth from standing before the Lord of all the earth."],
['Zechariah','Zech','06','006','6',"The black horses which \emph{are} therein go forth into the north country; and the white go forth after them; and the grisled go forth toward the south country."],
['Zechariah','Zech','06','007','7',"And the bay went forth, and sought to go that they might walk to and fro through the earth: and he said, Get you hence, walk to and fro through the earth. So they walked to and fro through the earth."],
['Zechariah','Zech','06','008','8',"Then cried he upon me, and spake unto me, saying, Behold, these that go toward the north country have quieted my spirit in the north country."],
['Zechariah','Zech','06','009','9',"And the word of the LORD came unto me, saying,"],
['Zechariah','Zech','06','010','10',"Take of \emph{them} \emph{of} the captivity, \emph{even} of Heldai, of Tobijah, and of Jedaiah, which are come from Babylon, and come thou the same day, and go into the house of Josiah the son of Zephaniah;"],
['Zechariah','Zech','06','011','11',"Then take silver and gold, and make crowns, and set \emph{them} upon the head of Joshua the son of Josedech, the high priest;"],
['Zechariah','Zech','06','012','12',"And speak unto him, saying, Thus speaketh the LORD of hosts, saying, Behold the man whose name \emph{is} The BRANCH; and he shall grow up out of his place, and he shall build the temple of the LORD:"],
['Zechariah','Zech','06','013','13',"Even he shall build the temple of the LORD; and he shall bear the glory, and shall sit and rule upon his throne; and he shall be a priest upon his throne: and the counsel of peace shall be between them both."],
['Zechariah','Zech','06','014','14',"And the crowns shall be to Helem, and to Tobijah, and to Jedaiah, and to Hen the son of Zephaniah, for a memorial in the temple of the LORD."],
['Zechariah','Zech','06','015','15',"And they \emph{that} \emph{are} far off shall come and build in the temple of the LORD, and ye shall know that the LORD of hosts hath sent me unto you. And \emph{this} shall come to pass, if ye will diligently obey the voice of the LORD your God."]
]

Zechariah07 = [
    ['Zechariah','Zech','07','001','1',"And it came to pass in the fourth year of king Darius, \emph{that} the word of the LORD came unto Zechariah in the fourth \emph{day} of the ninth month, \emph{even} in Chisleu;"],
['Zechariah','Zech','07','002','2',"When they had sent unto the house of God Sherezer and Regem-melech, and their men, to pray before the LORD,"],
['Zechariah','Zech','07','003','3',"\emph{And} to speak unto the priests which \emph{were} in the house of the LORD of hosts, and to the prophets, saying, Should I weep in the fifth month, separating myself, as I have done these so many years?"],
['Zechariah','Zech','07','004','4',"Then came the word of the LORD of hosts unto me, saying,"],
['Zechariah','Zech','07','005','5',"Speak unto all the people of the land, and to the priests, saying, When ye fasted and mourned in the fifth and seventh \emph{month}, even those seventy years, did ye at all fast unto me, \emph{even} to me?"],
['Zechariah','Zech','07','006','6',"And when ye did eat, and when ye did drink, did not ye eat \emph{for} \emph{yourselves}, and drink \emph{for} \emph{yourselves}?"],
['Zechariah','Zech','07','007','7',"\emph{Should} \emph{ye} not \emph{hear} the words which the LORD hath cried by the former prophets, when Jerusalem was inhabited and in prosperity, and the cities thereof round about her, when \emph{men} inhabited the south and the plain?"],
['Zechariah','Zech','07','008','8',"And the word of the LORD came unto Zechariah, saying,"],
['Zechariah','Zech','07','009','9',"Thus speaketh the LORD of hosts, saying, Execute true judgment, and shew mercy and compassions every man to his brother:"],
['Zechariah','Zech','07','010','10',"And oppress not the widow, nor the fatherless, the stranger, nor the poor; and let none of you imagine evil against his brother in your heart."],
['Zechariah','Zech','07','011','11',"But they refused to hearken, and pulled away the shoulder, and stopped their ears, that they should not hear."],
['Zechariah','Zech','07','012','12',"Yea, they made their hearts \emph{as} an adamant stone, lest they should hear the law, and the words which the LORD of hosts hath sent in his spirit by the former prophets: therefore came a great wrath from the LORD of hosts."],
['Zechariah','Zech','07','013','13',"Therefore it is come to pass, \emph{that} as he cried, and they would not hear; so they cried, and I would not hear, saith the LORD of hosts:"],
['Zechariah','Zech','07','014','14',"But I scattered them with a whirlwind among all the nations whom they knew not. Thus the land was desolate after them, that no man passed through nor returned: for they laid the pleasant land desolate."]
]

Zechariah08 = [
    ['Zechariah','Zech','08','001','1',"Again the word of the LORD of hosts came \emph{to} \emph{me}, saying,"],
    ['Zechariah','Zech','08','002','2',"Thus saith the LORD of hosts; I was jealous for Zion with great jealousy, and I was jealous for her with great fury."],
    ['Zechariah','Zech','08','003','3',"Thus saith the LORD; I am returned unto Zion, and will dwell in the midst of Jerusalem: and Jerusalem shall be called a city of truth; and the mountain of the LORD of hosts the holy mountain."],
    ['Zechariah','Zech','08','004','4',"Thus saith the LORD of hosts; There shall yet old men and old women dwell in the streets of Jerusalem, and every man with his staff in his hand for very age."],
    ['Zechariah','Zech','08','005','5',"And the streets of the city shall be full of boys and girls playing in the streets thereof."],
    ['Zechariah','Zech','08','006','6',"Thus saith the LORD of hosts; If it be marvellous in the eyes of the remnant of this people in these days, should it also be marvellous in mine eyes? saith the LORD of hosts."],
    ['Zechariah','Zech','08','007','7',"Thus saith the LORD of hosts; Behold, I will save my people from the east country, and from the west country;"],
    ['Zechariah','Zech','08','008','8',"And I will bring them, and they shall dwell in the midst of Jerusalem: and they shall be my people, and I will be their God, in truth and in righteousness."],
    ['Zechariah','Zech','08','009','9',"Thus saith the LORD of hosts; Let your hands be strong, ye that hear in these days these words by the mouth of the prophets, which \emph{were} in the day \emph{that} the foundation of the house of the LORD of hosts was laid, that the temple might be built."],
    ['Zechariah','Zech','08','010','10',"For before these days there was no hire for man, nor any hire for beast; neither \emph{was} \emph{there} \emph{any} peace to him that went out or came in because of the affliction: for I set all men every one against his neighbour."],
    ['Zechariah','Zech','08','011','11',"But now I \emph{will} not \emph{be} unto the residue of this people as in the former days, saith the LORD of hosts."],
    ['Zechariah','Zech','08','012','12',"For the seed \emph{shall} \emph{be} prosperous; the vine shall give her fruit, and the ground shall give her increase, and the heavens shall give their dew; and I will cause the remnant of this people to possess all these \emph{things}."],
    ['Zechariah','Zech','08','013','13',"And it shall come to pass, \emph{that} as ye were a curse among the heathen, O house of Judah, and house of Israel; so will I save you, and ye shall be a blessing: fear not, \emph{but} let your hands be strong."],
    ['Zechariah','Zech','08','014','14',"For thus saith the LORD of hosts; As I thought to punish you, when your fathers provoked me to wrath, saith the LORD of hosts, and I repented not:"],
    ['Zechariah','Zech','08','015','15',"So again have I thought in these days to do well unto Jerusalem and to the house of Judah: fear ye not."],
    ['Zechariah','Zech','08','016','16',"These \emph{are} the things that ye shall do; Speak ye every man the truth to his neighbour; execute the judgment of truth and peace in your gates:"],
    ['Zechariah','Zech','08','017','17',"And let none of you imagine evil in your hearts against his neighbour; and love no false oath: for all these \emph{are} \emph{things} that I hate, saith the LORD."],
    ['Zechariah','Zech','08','018','18',"And the word of the LORD of hosts came unto me, saying,"],
    ['Zechariah','Zech','08','019','19',"Thus saith the LORD of hosts; The fast of the fourth \emph{month}, and the fast of the fifth, and the fast of the seventh, and the fast of the tenth, shall be to the house of Judah joy and gladness, and cheerful feasts; therefore love the truth and peace."],
    ['Zechariah','Zech','08','020','20',"Thus saith the LORD of hosts; \emph{It} \emph{shall} yet \emph{come} \emph{to} \emph{pass}, that there shall come people, and the inhabitants of many cities:"],
    ['Zechariah','Zech','08','021','21',"And the inhabitants of one \emph{city} shall go to another, saying, Let us go speedily to pray before the LORD, and to seek the LORD of hosts: I will go also."],
    ['Zechariah','Zech','08','022','22',"Yea, many people and strong nations shall come to seek the LORD of hosts in Jerusalem, and to pray before the LORD."],
    ['Zechariah','Zech','08','023','23',"Thus saith the LORD of hosts; In those days \emph{it} \emph{shall} \emph{come} \emph{to} \emph{pass}, that ten men shall take hold out of all languages of the nations, even shall take hold of the skirt of him that is a Jew, saying, We will go with you: for we have heard \emph{that} God \emph{is} with you."]
]

Zechariah09 = [
    ['Zechariah','Zech','09','001','1',"The burden of the word of the LORD in the land of Hadrach, and Damascus \emph{shall} \emph{be} the rest thereof: when the eyes of man, as of all the tribes of Israel, \emph{shall} \emph{be} toward the LORD."],
['Zechariah','Zech','09','002','2',"And Hamath also shall border thereby; Tyrus, and Zidon, though it be very wise."],
['Zechariah','Zech','09','003','3',"And Tyrus did build herself a strong hold, and heaped up silver as the dust, and fine gold as the mire of the streets."],
['Zechariah','Zech','09','004','4',"Behold, the Lord will cast her out, and he will smite her power in the sea; and she shall be devoured with fire."],
['Zechariah','Zech','09','005','5',"Ashkelon shall see \emph{it}, and fear; Gaza also \emph{shall} \emph{see} \emph{it}, and be very sorrowful, and Ekron; for her expectation shall be ashamed; and the king shall perish from Gaza, and Ashkelon shall not be inhabited."],
['Zechariah','Zech','09','006','6',"And a bastard shall dwell in Ashdod, and I will cut off the pride of the Philistines."],
['Zechariah','Zech','09','007','7',"And I will take away his blood out of his mouth, and his abominations from between his teeth: but he that remaineth, even he, \emph{shall} \emph{be} for our God, and he shall be as a governor in Judah, and Ekron as a Jebusite."],
['Zechariah','Zech','09','008','8',"And I will encamp about mine house because of the army, because of him that passeth by, and because of him that returneth: and no oppressor shall pass through them any more: for now have I seen with mine eyes."],
['Zechariah','Zech','09','009','9',"Rejoice greatly, O daughter of Zion; shout, O daughter of Jerusalem: behold, thy King cometh unto thee: he \emph{is} just, and having salvation; lowly, and riding upon an ass, and upon a colt the foal of an ass."],
['Zechariah','Zech','09','010','10',"And I will cut off the chariot from Ephraim, and the horse from Jerusalem, and the battle bow shall be cut off: and he shall speak peace unto the heathen: and his dominion \emph{shall} \emph{be} from sea \emph{even} to sea, and from the river \emph{even} to the ends of the earth."],
['Zechariah','Zech','09','011','11',"As for thee also, by the blood of thy covenant I have sent forth thy prisoners out of the pit wherein \emph{is} no water."],
['Zechariah','Zech','09','012','12',"Turn you to the strong hold, ye prisoners of hope: even to day do I declare \emph{that} I will render double unto thee;"],
['Zechariah','Zech','09','013','13',"When I have bent Judah for me, filled the bow with Ephraim, and raised up thy sons, O Zion, against thy sons, O Greece, and made thee as the sword of a mighty man."],
['Zechariah','Zech','09','014','14',"And the LORD shall be seen over them, and his arrow shall go forth as the lightning: and the Lord GOD shall blow the trumpet, and shall go with whirlwinds of the south."],
['Zechariah','Zech','09','015','15',"The LORD of hosts shall defend them; and they shall devour, and subdue with sling stones; and they shall drink, \emph{and} make a noise as through wine; and they shall be filled like bowls, \emph{and} as the corners of the altar."],
['Zechariah','Zech','09','016','16',"And the LORD their God shall save them in that day as the flock of his people: for they \emph{shall} \emph{be} \emph{as} the stones of a crown, lifted up as an ensign upon his land."],
['Zechariah','Zech','09','017','17',"For how great \emph{is} his goodness, and how great \emph{is} his beauty! corn shall make the young men cheerful, and new wine the maids."]
]

Zechariah10 = [
        ['Zechariah','Zech','10','001','1',"Ask ye of the LORD rain in the time of the latter rain; \emph{so} the LORD shall make bright clouds, and give them showers of rain, to every one grass in the field."],
['Zechariah','Zech','10','002','2',"For the idols have spoken vanity, and the diviners have seen a lie, and have told false dreams; they comfort in vain: therefore they went their way as a flock, they were troubled, because \emph{there} \emph{was} no shepherd."],
['Zechariah','Zech','10','003','3',"Mine anger was kindled against the shepherds, and I punished the goats: for the LORD of hosts hath visited his flock the house of Judah, and hath made them as his goodly horse in the battle."],
['Zechariah','Zech','10','004','4',"Out of him came forth the corner, out of him the nail, out of him the battle bow, out of him every oppressor together."],
['Zechariah','Zech','10','005','5',"And they shall be as mighty \emph{men}, which tread down \emph{their} \emph{enemies} in the mire of the streets in the battle: and they shall fight, because the LORD \emph{is} with them, and the riders on horses shall be confounded."],
['Zechariah','Zech','10','006','6',"And I will strengthen the house of Judah, and I will save the house of Joseph, and I will bring them again to place them; for I have mercy upon them: and they shall be as though I had not cast them off: for I \emph{am} the LORD their God, and will hear them."],
['Zechariah','Zech','10','007','7',"And \emph{they} \emph{of} Ephraim shall be like a mighty \emph{man}, and their heart shall rejoice as through wine: yea, their children shall see \emph{it}, and be glad; their heart shall rejoice in the LORD."],
['Zechariah','Zech','10','008','8',"I will hiss for them, and gather them; for I have redeemed them: and they shall increase as they have increased."],
['Zechariah','Zech','10','009','9',"And I will sow them among the people: and they shall remember me in far countries; and they shall live with their children, and turn again."],
['Zechariah','Zech','10','010','10',"I will bring them again also out of the land of Egypt, and gather them out of Assyria; and I will bring them into the land of Gilead and Lebanon; and \emph{place} shall not be found for them."],
['Zechariah','Zech','10','011','11',"And he shall pass through the sea with affliction, and shall smite the waves in the sea, and all the deeps of the river shall dry up: and the pride of Assyria shall be brought down, and the sceptre of Egypt shall depart away."],
['Zechariah','Zech','10','012','12',"And I will strengthen them in the LORD; and they shall walk up and down in his name, saith the LORD."]
]

Zechariah11 = [
        ['Zechariah','Zech','11','001','1',"Open thy doors, O Lebanon, that the fire may devour thy cedars."],
['Zechariah','Zech','11','002','2',"Howl, fir tree; for the cedar is fallen; because the mighty are spoiled: howl, O ye oaks of Bashan; for the forest of the vintage is come down."],
['Zechariah','Zech','11','003','3',"\emph{There} \emph{is} a voice of the howling of the shepherds; for their glory is spoiled: a voice of the roaring of young lions; for the pride of Jordan is spoiled."],
['Zechariah','Zech','11','004','4',"Thus saith the LORD my God; Feed the flock of the slaughter;"],
['Zechariah','Zech','11','005','5',"Whose possessors slay them, and hold themselves not guilty: and they that sell them say, Blessed \emph{be} the LORD; for I am rich: and their own shepherds pity them not."],
['Zechariah','Zech','11','006','6',"For I will no more pity the inhabitants of the land, saith the LORD: but, lo, I will deliver the men every one into his neighbour's hand, and into the hand of his king: and they shall smite the land, and out of their hand I will not deliver \emph{them}."],
['Zechariah','Zech','11','007','7',"And I will feed the flock of slaughter, \emph{even} you, O poor of the flock. And I took unto me two staves; the one I called Beauty, and the other I called Bands; and I fed the flock."],
['Zechariah','Zech','11','008','8',"Three shepherds also I cut off in one month; and my soul lothed them, and their soul also abhorred me."],
['Zechariah','Zech','11','009','9',"Then said I, I will not feed you: that that dieth, let it die; and that that is to be cut off, let it be cut off; and let the rest eat every one the flesh of another."],
['Zechariah','Zech','11','010','10',"And I took my staff, \emph{even} Beauty, and cut it asunder, that I might break my covenant which I had made with all the people."],
['Zechariah','Zech','11','011','11',"And it was broken in that day: and so the poor of the flock that waited upon me knew that it \emph{was} the word of the LORD."],
['Zechariah','Zech','11','012','12',"And I said unto them, If ye think good, give \emph{me} my price; and if not, forbear. So they weighed for my price thirty \emph{pieces} of silver."],
['Zechariah','Zech','11','013','13',"And the LORD said unto me, Cast it unto the potter: a goodly price that I was prised at of them. And I took the thirty \emph{pieces} of silver, and cast them to the potter in the house of the LORD."],
['Zechariah','Zech','11','014','14',"Then I cut asunder mine other staff, \emph{even} Bands, that I might break the brotherhood between Judah and Israel."],
['Zechariah','Zech','11','015','15',"And the LORD said unto me, Take unto thee yet the instruments of a foolish shepherd."],
['Zechariah','Zech','11','016','16',"For, lo, I will raise up a shepherd in the land, \emph{which} shall not visit those that be cut off, neither shall seek the young one, nor heal that that is broken, nor feed that that standeth still: but he shall eat the flesh of the fat, and tear their claws in pieces."],
['Zechariah','Zech','11','017','17',"Woe to the idol shepherd that leaveth the flock! the sword \emph{shall} \emph{be} upon his arm, and upon his right eye: his arm shall be clean dried up, and his right eye shall be utterly darkened."]
]

Zechariah12 = [      
    ['Zechariah','Zech','12','001','1',"The burden of the word of the LORD for Israel, saith the LORD, which stretcheth forth the heavens, and layeth the foundation of the earth, and formeth the spirit of man within him."],
    ['Zechariah','Zech','12','002','2',"Behold, I will make Jerusalem a cup of trembling unto all the people round about, when they shall be in the siege both against Judah \emph{and} against Jerusalem."],
    ['Zechariah','Zech','12','003','3',"And in that day will I make Jerusalem a burdensome stone for all people: all that burden themselves with it shall be cut in pieces, though all the people of the earth be gathered together against it."],
    ['Zechariah','Zech','12','004','4',"In that day, saith the LORD, I will smite every horse with astonishment, and his rider with madness: and I will open mine eyes upon the house of Judah, and will smite every horse of the people with blindness."],
    ['Zechariah','Zech','12','005','5',"And the governors of Judah shall say in their heart, The inhabitants of Jerusalem \emph{shall} \emph{be} my strength in the LORD of hosts their God."],
    ['Zechariah','Zech','12','006','6',"In that day will I make the governors of Judah like an hearth of fire among the wood, and like a torch of fire in a sheaf; and they shall devour all the people round about, on the right hand and on the left: and Jerusalem shall be inhabited again in her own place, \emph{even} in Jerusalem."],
    ['Zechariah','Zech','12','007','7',"The LORD also shall save the tents of Judah first, that the glory of the house of David and the glory of the inhabitants of Jerusalem do not magnify \emph{themselves} against Judah."],
    ['Zechariah','Zech','12','008','8',"In that day shall the LORD defend the inhabitants of Jerusalem; and he that is feeble among them at that day shall be as David; and the house of David \emph{shall} \emph{be} as God, as the angel of the LORD before them."],
    ['Zechariah','Zech','12','009','9',"And it shall come to pass in that day, \emph{that} I will seek to destroy all the nations that come against Jerusalem."],
    ['Zechariah','Zech','12','010','10',"And I will pour upon the house of David, and upon the inhabitants of Jerusalem, the spirit of grace and of supplications: and they shall look upon me whom they have pierced, and they shall mourn for him, as one mourneth for \emph{his} only \emph{son}, and shall be in bitterness for him, as one that is in bitterness for \emph{his} firstborn."],
    ['Zechariah','Zech','12','011','11',"In that day shall there be a great mourning in Jerusalem, as the mourning of Hadadrimmon in the valley of Megiddon."],
    ['Zechariah','Zech','12','012','12',"And the land shall mourn, every family apart; the family of the house of David apart, and their wives apart; the family of the house of Nathan apart, and their wives apart;"],
    ['Zechariah','Zech','12','013','13',"The family of the house of Levi apart, and their wives apart; the family of Shimei apart, and their wives apart;"],
    ['Zechariah','Zech','12','014','14',"All the families that remain, every family apart, and their wives apart."],
    ['Zechariah','Zech','12','015','here',"shall be no more the Canaanite in the house of the LORD of hosts."]
]
Zechariah13 = [
    ['Zechariah','Zech','13','001','1',"In that day there shall be a fountain opened to the house of David and to the inhabitants of Jerusalem for sin and for uncleanness."],
    ['Zechariah','Zech','13','002','2',"And it shall come to pass in that day, saith the LORD of hosts, \emph{that} I will cut off the names of the idols out of the land, and they shall no more be remembered: and also I will cause the prophets and the unclean spirit to pass out of the land."],
    ['Zechariah','Zech','13','003','3',"And it shall come to pass, \emph{that} when any shall yet prophesy, then his father and his mother that begat him shall say unto him, Thou shalt not live; for thou speakest lies in the name of the LORD: and his father and his mother that begat him shall thrust him through when he prophesieth."],
    ['Zechariah','Zech','13','004','4',"And it shall come to pass in that day, \emph{that} the prophets shall be ashamed every one of his vision, when he hath prophesied; neither shall they wear a rough garment to deceive:"],
    ['Zechariah','Zech','13','005','5',"But he shall say, I \emph{am} no prophet, I \emph{am} an husbandman; for man taught me to keep cattle from my youth."],
    ['Zechariah','Zech','13','006','6',"And \emph{one} shall say unto him, What \emph{are} these wounds in thine hands? Then he shall answer, \emph{Those} with which I was wounded \emph{in} the house of my friends."],
    ['Zechariah','Zech','13','007','7',"Awake, O sword, against my shepherd, and against the man \emph{that} \emph{is} my fellow, saith the LORD of hosts: smite the shepherd, and the sheep shall be scattered: and I will turn mine hand upon the little ones."],
    ['Zechariah','Zech','13','008','8',"And it shall come to pass, \emph{that} in all the land, saith the LORD, two parts therein shall be cut off \emph{and} die; but the third shall be left therein."],
    ['Zechariah','Zech','13','009','9',"And I will bring the third part through the fire, and will refine them as silver is refined, and will try them as gold is tried: they shall call on my name, and I will hear them: I will say, It \emph{is} my people: and they shall say, The LORD \emph{is} my God."]
]

Zechariah14 = [
    ['Zechariah','Zech','14','001','1',"Behold, the day of the LORD cometh, and thy spoil shall be divided in the midst of thee."],
    ['Zechariah','Zech','14','002','2',"For I will gather all nations against Jerusalem to battle; and the city shall be taken, and the houses rifled, and the women ravished; and half of the city shall go forth into captivity, and the residue of the people shall not be cut off from the city."],
    ['Zechariah','Zech','14','003','3',"Then shall the LORD go forth, and fight against those nations, as when he fought in the day of battle."],
    ['Zechariah','Zech','14','004','4',"And his feet shall stand in that day upon the mount of Olives, which \emph{is} before Jerusalem on the east, and the mount of Olives shall cleave in the midst thereof toward the east and toward the west, \emph{and} \emph{there} \emph{shall} \emph{be} a very great valley; and half of the mountain shall remove toward the north, and half of it toward the south."],
    ['Zechariah','Zech','14','005','5',"And ye shall flee \emph{to} the valley of the mountains; for the valley of the mountains shall reach unto Azal: yea, ye shall flee, like as ye fled from before the earthquake in the days of Uzziah king of Judah: and the LORD my God shall come, \emph{and} all the saints with thee."],
    ['Zechariah','Zech','14','006','6',"And it shall come to pass in that day, \emph{that} the light shall not be clear, \emph{nor} dark:"],
    ['Zechariah','Zech','14','007','7',"But it shall be one day which shall be known to the LORD, not day, nor night: but it shall come to pass, \emph{that} at evening time it shall be light."],
    ['Zechariah','Zech','14','008','8',"And it shall be in that day, \emph{that} living waters shall go out from Jerusalem; half of them toward the former sea, and half of them toward the hinder sea: in summer and in winter shall it be."],
    ['Zechariah','Zech','14','009','9',"And the LORD shall be king over all the earth: in that day shall there be one LORD, and his name one."],
    ['Zechariah','Zech','14','010','10',"All the land shall be turned as a plain from Geba to Rimmon south of Jerusalem: and it shall be lifted up, and inhabited in her place, from Benjamin's gate unto the place of the first gate, unto the corner gate, and \emph{from} the tower of Hananeel unto the king's winepresses."],
    ['Zechariah','Zech','14','011','11',"And \emph{men} shall dwell in it, and there shall be no more utter destruction; but Jerusalem shall be safely inhabited."],
    ['Zechariah','Zech','14','012','12',"And this shall be the plague wherewith the LORD will smite all the people that have fought against Jerusalem; Their flesh shall consume away while they stand upon their feet, and their eyes shall consume away in their holes, and their tongue shall consume away in their mouth."],
    ['Zechariah','Zech','14','013','13',"And it shall come to pass in that day, \emph{that} a great tumult from the LORD shall be among them; and they shall lay hold every one on the hand of his neighbour, and his hand shall rise up against the hand of his neighbour."],
    ['Zechariah','Zech','14','014','14',"And Judah also shall fight at Jerusalem; and the wealth of all the heathen round about shall be gathered together, gold, and silver, and apparel, in great abundance."],
    ['Zechariah','Zech','14','015','15',"And so shall be the plague of the horse, of the mule, of the camel, and of the ass, and of all the beasts that shall be in these tents, as this plague."],
    ['Zechariah','Zech','14','016','16',"And it shall come to pass, \emph{that} every one that is left of all the nations which came against Jerusalem shall even go up from year to year to worship the King, the LORD of hosts, and to keep the feast of tabernacles."],
    ['Zechariah','Zech','14','017','17',"And it shall be, \emph{that} whoso will not come up of \emph{all} the families of the earth unto Jerusalem to worship the King, the LORD of hosts, even upon them shall be no rain."],
    ['Zechariah','Zech','14','018','18',"And if the family of Egypt go not up, and come not, that \emph{have} no \emph{rain}; there shall be the plague, wherewith the LORD will smite the heathen that come not up to keep the feast of tabernacles."],
    ['Zechariah','Zech','14','019','19',"This shall be the punishment of Egypt, and the punishment of all nations that come not up to keep the feast of tabernacles."],
    ['Zechariah','Zech','14','020','20',"In that day shall there be upon the bells of the horses, HOLINESS UNTO THE LORD; and the pots in the LORD'S house shall be like the bowls before the altar."],
    ['Zechariah','Zech','14','021','21',"Yea, every pot in Jerusalem and in Judah shall be holiness unto the LORD of hosts: and all they that sacrifice shall come and take of them, and seethe therein: and in that day there shall be no more the Canaanite in the house of the LORD of hosts."]
]

def main():
    ProcessVerses('Zechariah13',Zechariah13)
      
if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{In that day there shall be a fountain opened to the house of David and to the inhabitants of Jerusalem for sin and for uncleanness.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And it shall come to pass in that day, saith the LORD of hosts, \emph{that} I will cut off the names of the idols out of the land, and they shall no more be remembered: and also I will cause the prophets and the unclean spirit to pass out of the land.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And it shall come to pass, \emph{that} when any shall yet prophesy, then his father and his mother that begat him shall say unto him, Thou shalt not live; for thou speakest lies in the name of the LORD: and his father and his mother that begat him shall thrust him through when he prophesieth.}
[4] \textcolor[cmyk]{0.99998,1,0,

house & 2 \\ \hline
husbandman & 1 \\ \hline
idols & 1 \\ \hline
in & 5 \\ \hline
inhabitants & 1 \\ \hline
is & 2 \\ \hline
it & 4 \\ \hline
keep & 1 \\ \hline
land & 3 \\ \hline
left & 1 \\ \hline
lies & 1 \\ \hline
little & 1 \\ \hline
live & 1 \\ \hline
man & 2 \\ \hline
me & 1 \\ \hline
mine & 1 \\ \hline
more & 1 \\ \hline
mother & 2 \\ \hline
my & 7 \\ \hline
name & 2 \\ \hline
names & 1 \\ \hline
neither & 1 \\ \hline
no & 2 \\ \hline
not & 1 \\ \hline
of & 10 \\ \hline
off & 2 \\ \hline
on & 1 \\ \hline
one & 1 \\ \hline
ones & 1 \\ \hline
opened & 1 \\ \hline
out & 2 \\ \hline
part & 1 \\ \hline
parts & 1 \\ \hline
pass & 5 \\ \hline
people & 1 \\ \hline
prophesied & 1 \\ \hline
prophesieth & 1 \\ \hline
prophesy & 1 \\ \hline
prophet & 1 \\ \hline
prophets & 2 \\ \hline
refine & 1 \\ \hline
refined & 1 \\ \hline
remembered & 1 \\ \hline
rough & 1 \\ \hline
saith & 3 \\ \hline
say & 5 \\ \hline
scattered & 1 \\ \hline
shall & 19 \\ \hline
shalt & 1 \\ \hline
sheep & 1 \\ \hli